In [91]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


In [92]:
# read the file
# mortality
zipped = True
if zipped:
    mdf_path = r'data\mdf_final.csv'
    mdf = pd.read_csv(mdf_path, low_memory=False)
    mdf = mdf.drop(columns='Unnamed: 0')
else:
    mortality_path_7 = r'data/MortIcd7.csv'
    mortality_data_7 = pd.read_csv(mortality_path_7, low_memory=False)

    mortality_path_8 = r'data/MortIcd8.csv'
    mortality_data_8 = pd.read_csv(mortality_path_8, low_memory=False)

    mortality_path_9 = r'data/Morticd9.csv'
    mortality_data_9 = pd.read_csv(mortality_path_9, low_memory=False)

    mortality_path_2002 = r'data/Morticd10_part1.csv'
    mortality_data_2002 = pd.read_csv(mortality_path_2002, low_memory=False)

    mortality_path_2007 = r'data/Morticd10_part2.csv'
    mortality_data_2007 = pd.read_csv(mortality_path_2007, low_memory=False)

    mortality_path_2012 = r'data/Morticd10_part3.csv'
    mortality_data_2012 = pd.read_csv(mortality_path_2012, low_memory=False)

    mortality_path_2016 = r'data/Morticd10_part4.csv'
    mortality_data_2016 = pd.read_csv(mortality_path_2016, low_memory=False)

    mdf = pd.concat([mortality_data_7, mortality_data_8, mortality_data_9, mortality_data_2002, mortality_data_2007, mortality_data_2012, mortality_data_2016], ignore_index=True)

# population
pop_path = r'data/pop.csv'
pdf = pd.read_csv(pop_path, low_memory=False)

# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [93]:
# working, deleted the not useful columns
dropped = True
if not dropped:
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)
    pdf = pdf.drop(columns = undesired_population_columns)

In [94]:
# needs to swap out the nah columns into averages 
done = True
if not done:
    not_used_country_list = []
    country_num_count = 0
    not_useful = False
    for key in country_code_map:
        country_num_count += 1
        display(country_num_count / len(country_code_map))
        country_mdf = mdf[mdf['Country'] == key]
        
        if country_mdf.empty:
            not_used_country_list.append(key)
        
        else:
            min_year = int(country_mdf['Year'].min())
            max_year = int(country_mdf['Year'].max()) 
            for i in range (min_year, max_year + 1):
                
                if not_useful:
                    break
                
                mask = (mdf['Country'] == key) & (mdf['Year'] == i)
                curr_Year = i

                for c in mdf.columns.values:
                    
                    if not_useful:
                        break
                    
                    average = mdf.loc[mask, c].mean()   
                    
                    max_iter = max_year - curr_Year
                    count = 0 
                    while np.isnan(average):
                        curr_Year += 1
                        count += 1
                        mask_alt = (mdf['Country'] == key) & (mdf['Year'] == curr_Year) 
                        average = mdf.loc[mask_alt, c].mean()   
                        
                        if count > max_iter:
                            not_useful = True
                            break 
                        
                        
                    mdf.loc[mask, c] = mdf.loc[mask, c].fillna(average)
                    curr_Year = i 
                    
            if not_useful:
                display(key)
                not_used_country_list.append(key)
                index = mdf[mdf['Country'] == key].index
                mdf = mdf.drop(index)
                not_useful = False
                
    undesired_mortality_columns = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'] 
    mdf = mdf.drop(columns = undesired_mortality_columns)            
    mdf = mdf.groupby(['Country', 'Year'], as_index=True).agg("sum").reset_index()
    mdf.to_csv(r'data/mdf_final.csv')
            

        

In [96]:
pdf.isnull().sum()

Country       0
Admin1     9917
SubDiv     9857
Year          0
Sex           0
Frmat         0
Pop1          0
Pop2        136
Pop3        136
Pop4       4255
Pop5       4255
Pop6       4255
Pop7        136
Pop8        154
Pop9        136
Pop10       154
Pop11       136
Pop12       154
Pop13       136
Pop14       154
Pop15       136
Pop16       154
Pop17       136
Pop18       154
Pop19       136
Pop20       198
Pop21       292
Pop22      1206
Pop23      1206
Pop24      8468
Pop25      8468
Pop26       154
Lb          316
dtype: int64

In [98]:
dropped = True
if not dropped:
    undesired_population_columns = ['Admin1', 'SubDiv', 'Sex', 'Frmat'] 
    pdf = pdf.drop(columns = undesired_population_columns)
pdf

,Country,Year,Pop1,Pop2,Pop3,Pop4,Pop5,Pop6,Pop7,Pop8,...,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26,Lb
0,1060,1980,137100.0,3400.0,15800.0,NaN,NaN,NaN,39200.0,NaN,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,1980,159000.0,4000.0,18400.0,NaN,NaN,NaN,45400.0,NaN,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0
2,1125,1955,5051500.0,150300.0,543400.0,NaN,NaN,NaN,615300.0,603000.0,...,110200.0,51100.0,41600.0,14300.0,11800.0,25300.0,NaN,NaN,0.0,253329.0
3,1125,1955,5049400.0,145200.0,551000.0,NaN,NaN,NaN,613400.0,600700.0,...,122100.0,51100.0,50700.0,15800.0,18000.0,28500.0,NaN,NaN,0.0,237901.0
4,1125,1956,5353700.0,158700.0,576600.0,NaN,NaN,NaN,653000.0,639400.0,...,116900.0,54100.0,44000.0,14900.0,12400.0,26600.0,NaN,NaN,0.0,250022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,4140,2020,5492836.0,41382.0,42532.0,44452.0,46788.0,47730.0,245927.0,269859.0,...,364453.0,325056.0,308424.0,243412.0,229435.0,147031.0,58083.0,20861.0,0.0,41233.0
9995,4190,2021,322260.0,3357.0,3352.0,3392.0,3477.0,3454.0,17706.0,17410.0,...,18385.0,14120.0,11255.0,7803.0,5293.0,2879.0,1144.0,164.0,0.0,3417.0
9996,4190,2021,317804.0,3224.0,3269.0,3323.0,3307.0,3338.0,16810.0,16619.0,...,17758.0,14607.0,11935.0,8984.0,7502.0,4996.0,2549.0,740.0,0.0,3273.0
9997,4182,2021,9224590.0,223552.0,212137.0,204028.0,200720.0,201373.0,982270.0,857617.0,...,375734.0,241436.0,149026.0,60581.0,57026.0,17029.0,5915.0,3983.0,0.0,230272.0


In [99]:
country_list = list(set(mdf['Country']))
country_list.sort()
len(country_list) 

103

In [88]:
key = 1060
country_pdf = pdf[pdf['Country'] == key]
display(country_pdf)
country_pdf.isnull().sum().sum()


,Country,Admin1,SubDiv,Year,Sex,Frmat,Pop1,Pop2,Pop3,Pop4,...,Pop18,Pop19,Pop20,Pop21,Pop22,Pop23,Pop24,Pop25,Pop26,Lb
0,1060,NaN,NaN,1980,1,7,137100.0,3400.0,15800.0,NaN,...,NaN,5300.0,NaN,2900.0,NaN,NaN,NaN,NaN,6500.0,5000.0
1,1060,NaN,NaN,1980,2,7,159000.0,4000.0,18400.0,NaN,...,NaN,6200.0,NaN,3400.0,NaN,NaN,NaN,NaN,7500.0,6000.0


32

In [89]:
not_used_pop_list = []
country_num_count = 0
not_useful = False
for key in country_list:
    
        country_num_count += 1
        display(country_num_count / len(country_list))
        
        country_pdf = pdf[pdf['Country'] == key]
        
        if country_pdf.empty:
            not_used_pop_list.append(key)
        
        else:
            min_year = int(country_pdf['Year'].min())
            max_year = int(country_pdf['Year'].max()) 
            
            if min_year == max_year:
                if (country_pdf.isnull().sum().sum() > 0):
                    not_useful = True
                else:
                    continue
            
            for i in range (min_year, max_year + 1):
                
                if not_useful:
                    break
                
                mask = (pdf['Country'] == key) & (pdf['Year'] == i)
                curr_Year = i

                for c in pdf.columns.values:
                    
                    if not_useful:
                        break
                    
                    average = pdf.loc[mask, c].mean()   
                    
                    max_iter = max_year - curr_Year
                    count = 0 
                    while np.isnan(average):
                        curr_Year += 1
                        count += 1
                        mask_alt = (pdf['Country'] == key) & (pdf['Year'] == curr_Year)
                        average = pdf.loc[mask_alt, c].mean()   
                        
                        if count > max_iter:
                            not_useful = True
                            break 
                        
                        
                    pdf.loc[mask, c] = pdf.loc[mask, c].fillna(average)
                    curr_Year = i 
                    
            if not_useful:
                display(key)
                not_used_pop_list.append(key)
                index = pdf[pdf['Country'] == key].index
                pdf = pdf.drop(index)
                not_useful = False
    

0.009708737864077669

1125

0.019417475728155338

1300

0.02912621359223301

...

4190

0.7766990291262136

4195

0.7864077669902912

KeyboardInterrupt: 

In [ ]:
pdf.isnull().sum()

Country       0
Year          0
Pop1          0
Pop2         60
Pop3         60
Pop4       1271
Pop5       1271
Pop6       1271
Pop7         60
Pop8         72
Pop9         60
Pop10        72
Pop11        60
Pop12        72
Pop13        60
Pop14        72
Pop15        60
Pop16        72
Pop17        60
Pop18        72
Pop19        60
Pop20        76
Pop21       138
Pop22       342
Pop23       342
Pop24      2352
Pop25      2352
Pop26        60
Lb          186
dtype: int64

In [ ]:
for key in country_code_map:
    country_mdf = mdf[mdf['Country'] == key]
    display(country_mdf)

In [ ]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause', 'IM_Deaths1','IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4']
modified_dict = {}
for i in mdf.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []
consider_population = True

for key in country_code_map: 
    temp_mortality_df = mdf[mdf['Country'] == key] # filter to the country
    temp_pop_data = pop_data[pop_data['Country'] == key]
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            temp_temp_pop_data = temp_pop_data.loc[temp_pop_data['Year'] == i] 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                k = 1
                for c in column_name[2:]:
                    j = f'Pop{k}'
                    population = temp_temp_pop_data[j].sum()
                    if c in temp_temp_mortality_df.columns.values:
                        if consider_population:
                            if population == 0:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = 0
                            else:
                                modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() / population
                        else:    
                            modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum() 
                    k += 1
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()      

In [ ]:
# delete keys from dictionary
for key in list_delete_keys:
    del country_code_map[key]

In [ ]:
# building the mortality model 
country_mortality_model = {}
final_beginning_year = 9999
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    # temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = temp_modified_df['Year'].max()
    
    if last_year < final_beginning_year:
        final_beginning_year = last_year
    
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year + 1
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
    aphla = (last_row - first_row) / duration

    country_mortality_model[key] = {'a': aphla, 'd': duration}

In [ ]:
final_ending_year = 2050
current_year = 2000

mortality_pred_all = []
while current_year <= final_ending_year:
    
    mortality_pred_year = {}
    
    for key in country_code_map: 
        
        print(key)
        
        # isolate the data to the target data row \n",
        temp_modified_df = modified_df[(modified_df['Country'] == key)]  
        # check if there is real data for this year for this country\n",
        if current_year <= 2002:
            df = temp_modified_df[temp_modified_df['Year'] == current_year + 1]
            if not df.empty:
                df = df.loc[:, ~df.columns.isin(['Country', 'Year'])].values.astype('float')
                mortality_pred_year[key] = df 
                continue
 
        # clean the data for the row
        temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
        # find the aphla, b and k value for the model for future \n",
        aphla = country_mortality_model[key]['a']
        duration = country_mortality_model[key]['d']
        m_hat = (temp_modified_df - aphla)
        s, u, v = np.linalg.svd(m_hat)
        scalar = s[0]
        k = scalar * u[0]
        b = v[0] / scalar 
  
        # compute the next years result",
        change = np.exp(aphla + b * k)
        display(temp_modified_df.shape)
        display(change.shape)
        next_year = temp_mortality_df + change
        mortality_pred_year[key] = next_year
        
        # update parameters
        duration += 1
        aphla = (aphla * duration + change) / duration
        country_mortality_model[key]['a'] = aphla
        country_mortality_model[key]['d'] = duration
    
    current_year += 1
    mortality_pred_all.append(mortality_pred_year)
        
        


In [ ]:
key = 1125
temp_modified_df = modified_df[(modified_df['Country'] == key)]  
# clean the data for the row
temp_modified_df = temp_modified_df[temp_modified_df['Year'] == current_year].loc[:, ~temp_modified_df.columns.isin(['Country', 'Year'])].values.astype('float')
# find the aphla, b and k value for the model for future \n",
aphla = country_mortality_model[key]['a']
duration = country_mortality_model[key]['d']
m_hat = (temp_modified_df - aphla)
s, u, v = np.linalg.svd(m_hat)
scalar = s[0]
k = scalar * u[0]
b = v[0] / scalar 

display(aphla)
display(k, b)
change = np.exp(aphla + b * k)
display(temp_modified_df)
display(change)
display(temp_modified_df.shape)
next_year = temp_mortality_df + change
mortality_pred_year[key] = next_year